# expohack_kaktusi_18
Repo for hackathonsrus

![2024-08-03 12 03 29](https://github.com/user-attachments/assets/46f6b994-8ef2-4503-aaf7-ed2c7c67bce7)


Идея проекта основана на обучении ML модели (выдача релевантных доп. продуктов на основе портрета пользователя и тесных связей между продуктами Экспобанка и партнеров) и интеграции этой системы в инструменты работы менеджера по продажам (CRM, 1C и пр.) для автоматизации процессов.

# Схема работы системы

### Тренировка ML модели
Тренировка происходит один раз в самом начале.
Ответственный сотрудник с определенными правами доступа, загружает в систему базу данных клиентов за последние 2-3 года (чем больше данных, тем точнее будут результаты работы модели).
Происходит обучение модели за счет скрытых связей между портретом клиента (оформленный продукт, тип занятости, доход, количество иждевенцев и пр.) и родственными связями между оформленным продуктом и иными продуктами банка и партнеров.

### Основная схема работы системы
В случае наличия CRM, наша система будет интегрирована в воронку продаж и автоматически собирать и отправлять данные клиента на сервис, тем самым позволяя системе работать автономно и не отвлекать менеджера на ручную проверку клиента.
После обработки, при выявлении потенциально интересного для клиента продукта, в зависимости от процессов компании, система создает новый лид в CRM, либо иным образом оповещает менеджера о релевантных продуктах в которых может быть заинтересован клиент.
В случае отсутствия CRM (или невозможности встраивания интеграции) будет реализована фронтенд-часть для ручной проверки клиента (загрузка файла, как пример, где содержимое файла - данные клиента в различных форматах).

Также будет реализована строгая система прав доступа к системе. Доступы к личным кабинетам будут выдаваться руководителями, а доступ к первоначальной тренировке модели и ее дообучению будет только у руководителя (или иное ответственное лицо), либо производиться командой разработки продукта.

Модель (в рамках полной версии) умеет дообучаться: системе, которая проработала какое-то время, приходит на вход обратная связь в виде списка клиентов, которым модель ранее порекомендовала продукты, и результат, приобрел ли клиент этот продукт (или заинтересовался компанией и приобрел смежный продукт в этой компании). На основе этого модель эволюционирует и дообучается.

### Коммуникация с клиентом
Для того, чтобы минимизировать количество необработанных релевантных лидов и недопроданных услуг, система, при выявлении релевантных продуктов, может отправлять клиенту сообщение в Telegram с кратким описанием услуги, которая ему может быть интересна. Эта опция срабатывает, если клиент дал согласие на отправку сообщений в этом виде (Telegram бот не засыпает клиентов сообщениями со всеми новинками банка и партнеров, а отправляет уведомления в случае нахождения потенциально интересных для клиента продуктов при каждом обновлении информации по клиенту в БД).
В случае, если есть возможность уведомить клиента о продукте через бота, система создает лид (либо отправляет менеджеру ответ во фронтенд части) с указанием того, что этому клиенту можно отправить данное сообщение, после чего менеджер сможет принять самостоятельное решение о способе коммуникации (в случае отправки сообщения через бота, мы фиксируем этот факт в системе и в случае обращения клиента по предложенному продукту, он уходит нужному менеджеру).
Чтобы повысить ценность Telegram-бота для клиента в него добавляется возможность регистрации и участие в реферальной системе группы компаний-партнеров Экспобанка и накопления "Экспобонусов", которые клиент может потратить на услуги внутри группы. При этом в момент направления сообщения с релевантным предложением бот может также информировать о возможных бонусах при этом.

Пример автоматической коммуникации с клиентом: клиент с высоким доходом забронировал люкс номер в отеле, но без доп. услуг, система это выявляет и отправляет клиенту сообщение в Telegram о том, что в отеле есть яхт-клуб или хороший ресторан, вместе с ссылкой на информацию о доп. услугах отеля.

Если менеджер отдела продаж пропустил и не обработал релевантный лид система оповещает об этом руководителя продаж и он может отдать этот лид другому менеджеру или обработать его на свое усмотрение.

Мы рассматривали автоматическую коммуникацию с клиентами через другие каналы связи, смс, эл. почта и пуш-системы, но пришли к выводу, что эти способы могут оказаться слишком дорогими и/или недоступными в России по ряду причин. Telegram бот в этом плане выглядит самым надежным и ненавязчивым способом оповещения.

## Схема работы прототипа проекта (MVP)
- Происходит проверка прав менеджера
- На вход системе поступают данные о клиенте (в формате JSON)
- Происходит валидация данных: проверка наличия важных полей для работы ML модели, проверка наличия пользователя в БД для исключения лишней нагрузки на сервис (если такой пользователь уже был обработан, возвращается сохраненные данные обработки, при отсутствии дообучения модели с момента последней проверки)
- Данные пользователя сохраняются в системе (если их не было ранее)
- Происходит обработка данных клиента (ML)
- После получения результата работы ML модели, данные обрабатываются и упаковываются для дальнейшей отправки менеджеру
- Обработанные данные сохраняются в БД
- Происходит проверка наличия возможности отправки релевантного предложения через Telegram бота
- Происходит отправка данных менеджерам данной/других компаний (через интеграцию или на фронтенд)лю сообщение (пуш, смс или emai) с предложением релевантной услуги.
![photo_2024-08-06 12 01 21](https://github.com/user-attachments/assets/5d9c6359-3485-49c9-863b-b6e024e3fe52)
